# Decorators  in Python

## Python functions are objects

As usual, a Python function is a Python object

In [1]:
def f(x, y):
    pass

In [2]:
f.__class__

function

In [3]:
 issubclass(f.__class__, object)

True

This means for example that we can attach attributes to Python functions:

In [4]:
f.x = 2
f.__dict__

{'x': 2}

Use `dir` on any function to see the functions methods and attributes.

In [5]:
dir(f)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'x']

Calling the function invokes the `__call__` operator:

In [10]:
f(1, 2)

is the same as calling

In [11]:
f.__call__(1, 2)

## Functions as arguments

Like all objects, functions can be arguments to functions

In [12]:
def add(x,y):
    return x+y

def sub(x, y):
    return x-y

def apply(func, x, y):
    return func(x, y)

In [13]:
apply(add, 1, 2)

3

In [14]:
apply(sub, 7, 5)

2

## Functions inside functions

Python allows nested function definitions:

In [15]:
def g(x, y):
    
    def cube(x):
        return x*x*x
    
    return y*cube(x)

g(4, 6)

384

## Function returning functions

In [21]:
def h():
    pi = 0.13
    def inner_h():
        print("Inside inner_h but can access pi={}".format(pi))
        
    return inner_h

foo = h()
foo

<function __main__.h.<locals>.inner_h>

In [20]:
foo()

Inside inner_h but can access pi='0.13'


## More functions returning functions: *decorators*

A toy example

In [22]:
def foo():
    return 1

def outer(func):
    def inner():
        print("before calling func")
        return func()
    return inner

In [23]:
decorated = outer(foo)

The function `decorated` is a decorated version of function `foo`.
It is `foo` plus something more:

In [24]:
decorated()

before calling func


1

To simplify, we could just write
```python 
foo = outer(foo)
```
to replace foo with its decorated version each time it is called

## A (slightly) more useful decorator

Suppose we want to limit the range of values sent to a
mathematical formula:

In [25]:
from math import log
def f(x):
    return log(x) - 2

In [27]:
def checkrange(func):
    def inner(x):
        if x <= 0:
            print("out of range")
        else:
            return func(x)
    return inner

In [28]:
f(5)

-0.3905620875658997

In [29]:
f(-1)

ValueError: math domain error

In [30]:
f = checkrange(f)
f(5)

-0.3905620875658997

In [31]:
f(-1)

out of range


## The `@decorator` syntax

Python provides a short notation for decorating a function with
another function:

In [32]:
@checkrange
def g(x):
    return log(x) - 2

In [33]:
g(0)

out of range


This is essentially the same as writing `g = checkrange(g)`.

A decorator is simply a function taking another function as input
and returning another function. 

The syntax `@decorator` is a
short-cut for the more explicit `f = decorator(f)`.

## A (much) more useful decorator: memoization

Assume we have a slow function. Something like

In [34]:
from time import sleep

def slow(x, y):
    res = x*y; sleep(1)     # Simulate a long computation
    return res

In [35]:
%timeit slow(1, 2)

1 loop, best of 3: 1 s per loop


We call the function with the same input arguments, and hence perform the same (slow) calculations multiple times.

The idea of memoization (or buffering) is to buffer the input-output pairs for which the function was called.
If the function is called twice with same input arguments, we return the buffer value.

The implementation of a memoization with a `decorator` could look like:

In [36]:
cache = {}  # Stores all input-output pairs

def memoize(func):
    ''' Caches a function's return value each time it is called.
        If called later with the same arguments, the cached value is returned
        (not reevaluated). '''
    
    def inner(x, y):
        if (x, y) in cache:
            return cache[(x, y)]
        else:
            result = func(x, y)
            cache[(x, y)] = result
            return result
        
    return inner

Now we can apply the decorator to our slow function

In [37]:
@memoize
def slow(x, y):
    res = x*y; sleep(1)     # Simulate a long computation
    return res

... and test it out

In [38]:
%timeit -r 1 -n 1 slow(1, 3)

1 loop, best of 1: 1 s per loop


## Decorator summary 

* A function that takes a function as argument and returns a modified function
* `@decorator` syntax simply a short cut for the standard function call `f = decorator(f)`.